# Using Writer with AWS Strands Agents

This notebook demonstrates how to build an agent using the Strands Agents SDK with [WRITER’s models](https://dev.writer.com/home/integrations/strands#available-models).

Strands Agents SDK from AWS is an open-source framework that enables developers to build and deploy AI agents using a model-driven approach. This integration allows you to use Writer models within the Strands agent ecosystem, from local development to production deployment.

Support for running WRITER models through **Amazon Bedrock with Strands** is also available and is documented separately.


## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)


## Key concepts (how it works)

- Use Strands’ `Agent` class to create a self‑contained agent that interacts with a language model and optionally calls tools.
- WRITER’s Palmyra models (e.g., `palmyra-x5`, `palmyra-creative`) can be plugged into Strands via the WriterModel integration.
- In addition to direct WRITER API usage, the same Strands agent pattern can be used with **WRITER models hosted on Amazon Bedrock**.

## Setup
To use Writer models with Strands Agents, install the optional Writer dependency along with the [Strands Agents Tools package](https://github.com/strands-agents/tools):

In [ ]:
%pip install 'strands-agents[writer]'
%pip install strands-agents-tools

Next, set the `WRITER_API_KEY` environment variable. It is recommended to set this value in a `.env` file in the root of the project; however, this tutorial sets it directly as an environment variable if a `.env` file is not available.

In [4]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")


Enter your Writer API key:  ········


## Usage

Initialize a Strands agent using a Palmyra model from WRITER.

### Choose a Writer model

Import `WriterModel` from the Strands integration and specify a [model ID](https://dev.writer.com/home/integrations/strands#available-models) (for example, `palmyra‑x5`, `palmyra‑creative`, `palmyra‑med`). 


In [5]:
from strands import Agent
from strands.models.writer import WriterModel

writer_model = WriterModel(
    model_id="palmyra-x5"  # or other Palmyra model like palmyra-creative, palmyra-fin, etc.
)

### Configure tools

Define a list of tools to pass to the agent, such as a calculator or HTTP request tool.

> The `calculator` tool shown in this example is a prebuilt tool from the [`strands_tools` package](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/). You can use existing tools from `strands_tools`, or define custom tools—see the [Python Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/?h=python+tools) documentation for guidance.



In [ ]:
from strands_tools import calculator

tools = [calculator]


### Initialize the agent

Initialize the agent with the configured tools, `writer_model`, and a system prompt that defines the agent’s role, tone, output format, and how it should use tools.

> **Best practice:** Use a clear and specific system prompt to define your agent’s role and expected behavior. If you include tools, be explicit about when and how the agent should use them. For many use cases, a single agent with one model is sufficient and keeps the architecture simple.

In [ ]:
SYSTEM_PROMPT = """
You are a helpful enterprise assistant. Use your knowledge and available tools (if provided)
to answer user queries accurately and helpfully.
"""

agent = Agent(
    model=writer_model,
    tools=tools,         # or [] if no tools
    system_prompt=SYSTEM_PROMPT
)


### Run the agent

Once you’ve finished importing and initializing the WRITER provider in Strands Agents, you can ask the agent a question.

In [ ]:
response = agent("What is 2+2")
print(response)


## Example: advanced multi-agent research and content creation

This code provides an **extended example** of a multi-agent workflow 
using the Strands SDK. It goes beyond a basic demonstration to showcase a more 
detailed, real-world application scenario involving multiple specialized agents:

1. **Research Agent**: Leveraging `palmyra-x5` to perform comprehensive web-based research, 
   gather verified information, and summarize insights.

2. **Content Agent**: Utilizing `palmyra-creative` to craft engaging, platform-optimized 
   social media content based on the research findings.

This extended example covers:

- Importing and configuring necessary modules  
- Initializing and customizing models for each agent  
- Creating agents with specialized roles and workflows  
- Orchestrating a multi-step pipeline for research and content creation  
- Passing intermediate outputs between agents in a multi-step workflow

## Setup
Import the required modules to define agents, configure WRITER models, and enable tool-based HTTP requests:

In [ ]:
from strands import Agent
from strands.models.writer import WriterModel
from strands_tools import http_request

## Initialize the models

- `palmyra-creative` → for creative social content.
- `palmyra-x5` → for accurate research and factual lookups.


In [ ]:
# Initialize the creative content model
content_model = WriterModel(
    model_id="palmyra-creative",
)

# Initialize the research model
research_model = WriterModel(
    model_id="palmyra-x5",
)


## Define system prompts

- **Research agent prompt:** instructs the agent to gather and summarize factual information for a given topic.
- **X Post assistant prompt:** guides the agent to create an engaging social post tailored specifically for X/Twitter.
- **LinkedIn Post assistant prompt:** guides the agent to write a professional, value‑focused LinkedIn post.
- **Creative assistant prompt:** instructs the agent to produce flexible social content that can be adapted across platforms.
- **Style assistant prompt:** instructs the agent to rewrite existing content in a specified tone or style (e.g., professional, concise).


In [13]:
RESEARCH_ASSISTANT_PROMPT = """
You are the Research Assistant. Gather accurate factual information on the topic
provided and return a clear, structured summary of key findings, insights, and
important points relevant to the query.
"""

X_POST_ASSISTANT_PROMPT = """
You are the X Post Assistant. Using the input summary or topic, generate
short, engaging posts tailored specifically for X (Twitter). Focus on strong
hooks, concise language, and shareable insights in fewer than 280 characters.
"""

LINKEDIN_POST_ASSISTANT_PROMPT = """
You are the LinkedIn Post Assistant. Using the input summary or topic,
create a professional, value‑focused LinkedIn post. Include key takeaways,
industry relevance, and a call to engagement or reflection.
"""

CREATIVE_ASSISTANT_PROMPT = """
You are the Creative Content Assistant. Your job is to produce versatile
social content that may be repurposed for different platforms. Blend
original messaging with existing insights while maintaining clarity and tone.
"""


## Create agents

- **Research Agent** gathers and summarizes factual information about a topic using a Palmyra‑X5 model. It does *not* include live HTTP or web lookup tools in this example, but focuses on internal knowledge and reasoning.

- **X Post Assistant** generates short, engaging social posts optimized for X (formerly Twitter), using the Palmyra‑Creative model.

- **LinkedIn Post Assistant** produces value‑focused, professional content tailored specifically for LinkedIn, using the Palmyra‑Creative model.

- **Creative Assistant** provides versatile social content that can be adapted or repurposed across platforms when a general creative response is needed.

- **Style Assistant** (optional) restyles existing content into a requested tone or format (e.g., professional, concise, humorous).

All of these are defined as **tool‑wrapped agents** that the main orchestrator can call dynamically to satisfy different user needs.


In [14]:
from strands import tool

@tool
def research_assistant(query: str) -> str:
    """
    Research specialist: gathers and summarizes factual information for the topic.

    Args:
        query: The topic or question to research.
    Returns:
        A factual summary as text.
    """
    research_model = WriterModel(model_id="palmyra-x5")
    agent = Agent(
        model=research_model,
        system_prompt=RESEARCH_ASSISTANT_PROMPT,
        callback_handler=None,
    )
    return str(agent(query))


@tool
def x_post_assistant(query: str) -> str:
    """
    X Post specialist: crafts posts optimized for X (Twitter).

    Args:
        query: Summary or topic for the X post.
    Returns:
        A short, engaging X post.
    """
    x_model = WriterModel(model_id="palmyra-creative")
    agent = Agent(
        model=x_model,
        system_prompt=X_POST_ASSISTANT_PROMPT,
        callback_handler=None,
    )
    return str(agent(query))


@tool
def linkedin_post_assistant(query: str) -> str:
    """
    LinkedIn Post specialist: produces professional posts for LinkedIn.

    Args:
        query: Summary or topic for the LinkedIn post.
    Returns:
        A LinkedIn‑optimized post.
    """
    li_model = WriterModel(model_id="palmyra-creative")
    agent = Agent(
        model=li_model,
        system_prompt=LINKEDIN_POST_ASSISTANT_PROMPT,
        callback_handler=None,
    )
    return str(agent(query))


@tool
def creative_assistant(query: str) -> str:
    """
    Creative content specialist: generates flexible social content
    that can be adapted or repurposed for any platform.

    Args:
        query: Topic or research summary
    Returns:
        Versatile social media copy.
    """
    creative_model = WriterModel(model_id="palmyra-creative")
    agent = Agent(
        model=creative_model,
        system_prompt=CREATIVE_ASSISTANT_PROMPT,
        callback_handler=None,
    )
    return str(agent(query))


## Orchestrator Agent

This example uses a central **Orchestrator Agent** that coordinates multiple specialist agents, each wrapped as a tool. The orchestrator examines the user’s request and dynamically delegates subtasks to the appropriate assistant tools.

In [20]:
multi_agent_orchestrator = Agent(
    model=WriterModel(model_id="palmyra-x5"),
    system_prompt="""
You are the Orchestrator Agent. Decide whether to:
- call research_assistant(query) for facts,
- call x_post_assistant(summary) to generate an X post,
- call linkedin_post_assistant(summary) to generate a LinkedIn post,
- call creative_assistant(summary) for general creative social content.

Use tools explicitly and embed their results in your response.
""",
    callback_handler=None,
    tools=[
        research_assistant,
        x_post_assistant,
        linkedin_post_assistant,
        creative_assistant,
    ],
)


## Run the workflow

Call the orchestrator to research a topic and produce both an X/Twitter post and a LinkedIn post.

In [21]:
response = multi_agent_orchestrator(
    "Research sustainable fashion trends and produce both an X post and a LinkedIn post."
)
print(response)

Here are the results of your requests:

**X Post:**
"Sustainable fashion is on the rise! From circular business models to eco-friendly materials and second-hand shopping, consumers are driving change. Let's make fashion more sustainable! #SustainableFashion #EcoFriendly"

**LinkedIn Post:**
"The fashion industry is shifting towards sustainability, driven by consumer demand and technological innovation. Discover the key trends and opportunities in sustainable fashion.

The industry is adopting circular economy models, innovative materials, and ethical production practices. While there are challenges to overcome, the opportunities for growth and positive change are significant.

Some of the key trends include the use of eco-friendly materials, clothing rental services, and digital printing and 3D design technologies. Consumers are driving demand for sustainable fashion, and brands are responding by prioritizing transparency and sustainability.

As the industry continues to evolve, it's e

Research the topic and generate a concise, engaging post optimized for X.


In [24]:
response = multi_agent_orchestrator(
    "Research electric vehicles and create an engaging X post about the latest trends."
)
print(response)


Here is an engaging X post about the latest trends in electric vehicles:
"Electric vehicles are taking over the roads! With advancements in battery tech and expanded charging infrastructure, the future is electric! Share your favorite EV model and let's discuss the future of transportation! #ElectricVehicles #SustainableTransport"



Research the topic and produce a professional, insight‑driven LinkedIn post.

In [23]:
response = multi_agent_orchestrator(
    "Research remote work productivity and craft a professional LinkedIn post."
)
print(response)


Here is a professional LinkedIn post on remote work productivity:

"Boost your remote work productivity with these proven strategies and tools. From creating a conducive work environment to leveraging technology, discover how to stay focused, motivated, and connected while working remotely.

To optimize your remote work experience, consider the following:

* Create a dedicated workspace that is quiet, comfortable, and free from distractions
* Establish a structured schedule with regular working hours and breaks
* Minimize digital distractions and use tools like website blockers to stay on track
* Enhance communication and collaboration with your team through video conferencing and collaboration software
* Prioritize self-care and maintain a healthy work-life balance

Some popular productivity tools for remote workers include project management tools like Trello and Asana, time tracking tools like Toggl and Harvest, and communication tools like Slack and Zoom.

By implementing these str

## Using Strands with Amazon Bedrock

In addition to direct WRITER API integration, Strands also supports running WRITER models through **Amazon Bedrock**.

This is useful when you need:
- Centralized model access and governance through AWS
- Alignment with existing Bedrock-based infrastructure
- Standardized IAM-based authentication and auditing

When using Amazon Bedrock, the Strands agent architecture remains the same. The primary difference is the **model backend configuration**, which points to Bedrock-hosted WRITER models instead of the direct WRITER API.


### Bedrock Model Configuration

Instead of using `WriterModel`, use `BedrockModel` from Strands.

The following client configuration uses the `us.writer.palmyra-x5-v1:0` model ID.

Before selecting an inference profile, verify which AWS Regions currently support Writer models and ensure the chosen profile is enabled in your AWS account.

For credential configuration, Strands relies on the standard AWS credential resolution chain. Make sure your environment is configured with valid AWS credentials (for example, via environment variables, shared credentials files, or IAM roles) as described in the Strands [documentation](https://strandsagents.com/latest/documentation/docs/user-guide/quickstart/#configuring-credentials)

Additional details on enabling and using Writer models on Amazon Bedrock—including model availability, permissions, and setup—are covered in the Writer Bedrock integration [guide](https://dev.writer.com/home/integrations/bedrock)


Next, configure your AWS credentials for Amazon Bedrock access. It is recommended to set these values using standard AWS credential mechanisms (such as a `.env` file, shared credentials file, or IAM role). However, for demonstration purposes, this tutorial sets the credentials directly as environment variables if they are not already defined.


In [7]:
import getpass
import os

if not os.getenv("AWS_ACCESS_KEY_ID"):
    os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter your AWS_ACCESS_KEY_ID: ")

if not os.getenv("AWS_SECRET_ACCESS_KEY"):
    os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter your AWS_SECRET_ACCESS_KEY: ")

if not os.getenv("AWS_SESSION_TOKEN"):
    os.environ["AWS_SESSION_TOKEN"] = getpass.getpass("Enter your AWS_SESSION_TOKEN: ")

Enter your AWS_ACCESS_KEY_ID:  ········
Enter your AWS_SECRET_ACCESS_KEY:  ········
Enter your AWS_SESSION_TOKEN:  ········


### Common Bedrock configuration parameters

| Parameter | Description |
|----------|-------------|
| `model_id` | The Amazon Bedrock model identifier (for example, a WRITER Palmyra inference profile). |
| `temperature` | Controls response randomness; higher values produce more varied outputs. |
| `max_tokens` | Maximum number of tokens the model can generate in a single response. |
| `streaming` | Enables or disables streaming responses from the model. |
| `guardrail_id` | Identifier of the Amazon Bedrock guardrail to apply to model outputs. |
| `cache_prompt` | Enables caching of prompts to improve performance and reduce repeated computation. |
| `cache_tools` | Enables caching of tool definitions and usage for repeated requests. |
| `boto_session` | Custom `boto3` session used to control AWS credentials and configuration. |
| `additional_request_fields` | Additional model-specific parameters passed directly to the Bedrock request. |

<div style="
  border: 1px solid rgba(245, 158, 11, 0.2);
  background-color: rgba(245, 158, 11, 0.08);
  color: rgb(146, 64, 14);
  padding: 16px 20px;
  border-radius: 12px;
  margin: 16px 0;
">

<strong>Region resolution note</strong><br/>
When using Amazon Bedrock, the AWS region is resolved using boto3’s standard priority order.
If you experience unexpected region behavior, check your AWS configuration files and consider
explicitly setting <code>region_name</code> on the <code>BedrockModel</code>.

</div>



The example below shows how to configure a Strands agent to use a WRITER model hosted on Amazon Bedrock, using the `us.writer.palmyra-x5-v1:0` cross-region inference profile for Palmyra X5.  
Make sure the model is enabled in your AWS account and supported in your selected AWS Region.



In [10]:
from strands import Agent
from strands.models.bedrock import BedrockModel
from strands_tools import calculator

# Configure Amazon Bedrock model with Palmyra model ID
bedrock_model = BedrockModel(
    model_id="us.writer.palmyra-x5-v1:0",
    region_name="us-west-2",  # AWS region where the Bedrock model is enabled
)

agent = Agent(model=bedrock_model)
response = agent("Explain the benefits of using Bedrock + Strands for AI workflows.")
print(response)


Great question! Using Amazon Bedrock together with Strands creates a powerful combo for building and managing AI workflows—especially when you're aiming for both flexibility and enterprise-grade control.

Amazon Bedrock gives you serverless access to a range of top foundation models (FMs) from AWS and third-party providers like Anthropic, Meta, and Mistral. That means you can experiment with and deploy large language models without managing infrastructure—perfect for rapid prototyping and scaling.

Strands, on the other hand, is all about orchestration and governance. It helps you design, monitor, and manage complex AI workflows—chaining prompts, models, and tools—while adding visibility, versioning, and compliance controls. Think of it as a workflow engine tailored for LLM operations.

Together, they let you:
- Quickly swap and test models via Bedrock while using Strands to manage which model serves which use case.
- Enforce security, audit trails, and approval workflows across AI pip

## Example: Writer MCP + Strands agents   

This recipe demonstrates how to combine:

- **Writer MCP Server** (via `writer-sdk-mcp`)
- **Strands Agents**
- **WriterModel (Palmyra-x5)**
- **MCP Tools over stdio**

to orchestrate multi-step research and content generation workflows.


## Install dependencies

Install Strands (with Writer support) and the Strands Agent Tools package:

In [ ]:
%pip install 'strands-agents[writer]' strands-agents-tools

## Building a custom MCP client with Strands

In [29]:
import os
from strands import Agent
from strands.models.writer import WriterModel
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters



def log_server_init_and_requests(client_factory):
    """Wrap an MCP client to log server start and requests."""
    class LoggingMCPClient:
        def __init__(self):
            self._client = client_factory()

        def __enter__(self):
            print("[LOG] Starting MCP server...")
            self._client.__enter__()
            print("[LOG] MCP server started.")
            return self

        def __exit__(self, exc_type, exc_val, exc_tb):
            self._client.__exit__(exc_type, exc_val, exc_tb)

        def __getattr__(self, name):
            orig_attr = getattr(self._client, name)
            if callable(orig_attr):
                def hooked(*args, **kwargs):
                    print(f"[LOG] MCP request: {name}, args={args}, kwargs={kwargs}")
                    result = orig_attr(*args, **kwargs)
                    print(f"[LOG] MCP Server starting with tools: {',*\n'.join([el.tool_name for el in result])}")
                    return result
                return hooked
            return orig_attr
    return LoggingMCPClient()


# Connect to the Writer MCP server via stdio
writer_mcp_client = log_server_init_and_requests(lambda: MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="npx",
        args=["-y", "writer-sdk-mcp@latest"],
        env={"WRITER_API_KEY": os.environ["WRITER_API_KEY"]}
    )
)))

def content_workflow_client(user_request: str):
    """
    Custom MCP client that orchestrates complex content workflows
    using Writer MCP tools.
    """
    with writer_mcp_client:
        tools = writer_mcp_client.list_tools_sync()
        agent = Agent(
            model=writer_model,
            tools=tools,
            system_prompt="""You are a content workflow assistant that can:
            - Upload and analyze files
            - Generate content using Writer's models
            - Query Knowledge Graphs for research
            - Perform real-time web research for up-to-date, factual, or time-sensitive information using web_search_tools
            - Analyze images and extract insights
            - Chain multiple operations together seamlessly
            Use the available Writer MCP tools to fulfill user requests efficiently."""
        )
        return agent(user_request)


## Example workflow request

In [30]:
response = content_workflow_client(
    "Research and create content about sustainable fashion trends, "
    "including a blog post outline and social media content ideas."
)

print("Research & Content:", response)

[LOG] Starting MCP server...
[LOG] MCP server started.
[LOG] MCP request: list_tools_sync, args=(), kwargs={}
[LOG] MCP Server starting with tools: retrieve_applications,*
list_applications,*
generate_content_applications,*
create_applications_jobs,*
retrieve_applications_jobs,*
list_applications_jobs,*
retry_applications_jobs,*
update_applications_graphs,*
list_applications_graphs,*
chat_chat,*
create_completions,*
list_models,*
create_graphs,*
retrieve_graphs,*
update_graphs,*
list_graphs,*
delete_graphs,*
add_file_to_graph_graphs,*
question_graphs,*
remove_file_from_graph_graphs,*
retrieve_files,*
list_files,*
delete_files,*
download_files,*
retry_files,*
upload_files,*
ai_detect_tools,*
context_aware_splitting_tools,*
parse_pdf_tools,*
web_search_tools,*
medical_tools_comprehend,*
translate_translation,*
analyze_vision,*
search_docs
To create content about sustainable fashion trends, including a blog post outline and social media content ideas, we can break down the task into sever